In [3]:
import pytesseract
from PIL import Image
import cv2
import os,argparse
import matplotlib.pyplot as plt
import re
import ftfy
import sys
import json
import io

# Using regex to find the neceesary information
def findword(textlist, wordstring):
    lineno = -1
    for wordline in textlist:
        xx = wordline.split()
        if ([w for w in xx if re.search(wordstring, w)]):
            lineno = textlist.index(wordline)
            textlist = textlist[lineno+1:]
            return textlist
    return textlist

def voter_data(folder_dir):
    
    # Defining path to tesseract.exe and the image
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    
    DL = []
    
    for images in os.listdir(folder_dir):
        if (images.endswith(".png") or images.endswith(".jpg") or images.endswith(".jpeg")):
            inputPath = os.path.join(folder_dir, images)
            img = Image.open(inputPath)

            # Extract text from image
            text = pytesseract.image_to_string(img, lang ="eng")
            text = ftfy.fix_text(text)
            text = ftfy.fix_encoding(text)
            
            text1 = []
            # Splitting the lines to sort the text paragraph wise
            lines = text.split('\n')
            for lin in lines:
                s = lin.strip()
                s = s.rstrip()
                s = s.lstrip()
                DL.append(s)
            
            voter_no = findword(text1, '(ELECTION COMMISSION OF INDIA|CARD|IDENTITY CARD)$')
            voter_no = voter_no[0]
            voter_no = voter_no.replace(" ", "")
            
            lines = text
            
            for x in lines.split('\n'):
                _ = x.split()
                if ([w for w in _ if re.search("(Elector's|ELECTOR'S)$", w)]):    
                    person_name = x
                    person_name = person_name.split(':')[1].strip()
                    full_name = person_name
                
                if ([w for w in _ if re.search("(Father's|Mother's|FATHER'S|MOTHER'S)$", w)]):
                    elder_name = x
                    elder_name = elder_name.split(':')[1].strip()

                # Finding the gender of the electoral candidate
                if ([w for w in _ if re.search('(Male|MALE|male)$', w)]):
                    sex = "Male"
                elif ([w for w in _ if re.search('(Female|FEMALE|female)$', w)]):
                    sex = "Female"
                
                # Finding the Date of Birth 
                if ([w for w in _ if re.search('(Year|YEAR|Birth|Date|Date of Birth|DATE OF BIRTH|DOB)$', w)]):
                    dob = x
                    dob = dob.split(':')[1].strip()
            
            print(person_name)

folder_dir = r"C:\Users\VSK\Downloads\OCR\Voter"
DL = voter_data(folder_dir)
print(DL)

IndexError: list index out of range